In [1]:
import os
import time
import json
import argparse

import cv2
import numpy as np

from acl_resource import AclResource
from acl_model import Model

acl_resource = AclResource()
acl_resource.init()

import ipywidgets as widgets
from IPython.display import display

from DeepSortDetector import Detector

imgh, imgw = 720, 640
# https://blog.csdn.net/liuqixuan1994/article/details/88715454
imgbox = widgets.Image(format='jpg', height=imgh, width=imgw)
display(imgbox)


def read_class_names(ground_truth_json):
    with open(ground_truth_json, 'r') as file:
        content = file.read()
    content = json.loads(content)
    categories = content.get('categories')

    names = {}
    for id, category in enumerate(categories):
        category_name = category.get('name')
        if len(category_name.split()) == 2:
            temp = category_name.split()
            category_name = temp[0] + '_' + temp[1]
        names[id] = category_name.strip('\n')
    return names


def coco80_to_coco91_class():
    # converts 80-index (val2014/val2017) to 91-index (paper)
    x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34,
         35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
         64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]
    return x


def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=False, scaleFill=False, scaleup=True):
    # Resize image to a 32-pixel-multiple rectangle https://github.com/ultralytics/yolov3/issues/232
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, 64), np.mod(dh, 64)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)


def draw_bbox(bbox, img0, color, wt, names):
    det_result_str = ''
    for idx, class_id in enumerate(bbox[:, 5]):
        if float(bbox[idx][4] < float(0.05)):
            continue
        img0 = cv2.rectangle(img0, (int(bbox[idx][0]), int(bbox[idx][1])), (int(bbox[idx][2]), int(bbox[idx][3])),
                             color, wt)
        img0 = cv2.putText(img0, str(idx) + ' ' + names[int(class_id)], (int(bbox[idx][0]), int(bbox[idx][1] + 16)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        img0 = cv2.putText(img0, '{:.4f}'.format(bbox[idx][4]), (int(bbox[idx][0] + 64), int(bbox[idx][1] + 16)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        det_result_str += '{} {} {} {} {} {}\n'.format(
            names[bbox[idx][5]], str(bbox[idx][4]), bbox[idx][0], bbox[idx][1], bbox[idx][2], bbox[idx][3])
    return img0


def main():

    neth, netw = 640, 640

    ground_truth_json = '/home/HwHiAiUser/sample/dataset/annotations/instances_val2017.json'
    coco_names = read_class_names(ground_truth_json)
    coco91class = coco80_to_coco91_class()

    model_path = '/home/HwHiAiUser/sample/yolov5/output/yolov5s_v6.1_sim_nms_bs1.om'
    model = Model(acl_resource, model_path)

    cap = cv2.VideoCapture('/home/HwHiAiUser/sample/yolov5-v7.0/edge_infer/cars.mp4')
    fps = int(cap.get(5))
    videoWriter = None
    
    object_list = ['person', 'sports_ball', 'bicycle', 'motorcycle', 'car', 'bus', 'truck']
    shape = (neth, netw)
    conf_threshold = 0.1
    iou_threshold = 0.5
    names_file_path = 'coco_names.txt'
        
    deepsort_model_path = '/home/HwHiAiUser/sample/yolov5-v7.0/edge_infer/deep_dims.om'
    det = Detector(model, deepsort_model_path, neth, netw, object_list, shape, conf_threshold, iou_threshold, names_file_path)
    
    while True:
        # Read a picture from the camera
        ret, img_bgr = cap.read()  # YUV420SP NV12 8bit
        if not ret:
            break
        
        result = det.feedCap(img_bgr)
        result = result['frame']
        if videoWriter is None:
            fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')  # opencv3.0
            videoWriter = cv2.VideoWriter(
                f'result3.mp4', fourcc, fps, (result.shape[1], result.shape[0]))

        # gbox.value = cv2.imencode('.jpg', result)[1].tobytes()
        videoWriter.write(result)
            
if __name__ == '__main__':
    main()

[Sample] init resource stage:
Init resource success


Image(value=b'', format='jpg', height='720', width='640')

load model  /home/HwHiAiUser/sample/yolov5/output/yolov5s_v6.1_sim_nms_bs1.om
Init model resource
[Model] create model output dataset:
[Model] create model output dataset success
[Model] class Model init resource stage success


/home/HwHiAiUser/data/yolov5s-pretrained/infer_project/edge_infer/acl_model.py:151: Warning: acl.util.numpy_to_ptr will be deprecated. Please use acl.util.bytes_to_ptr instead.
  ptr = acl.util.numpy_to_ptr(input)
/home/HwHiAiUser/data/yolov5s-pretrained/infer_project/edge_infer/acl_model.py:234: Warning: acl.util.numpy_to_ptr will be deprecated. Please use acl.util.bytes_to_ptr instead.
  narray_ptr = acl.util.numpy_to_ptr(narray)
